<a href="https://colab.research.google.com/github/rahil404/User-Stories/blob/master/thesis_automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install build-essential swig
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn
# restart runtime after executing it

In [3]:
# import libraries
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import sklearn.metrics
import keras
from os import listdir
import numpy as np
import pandas as pd
from PIL import Image
import joblib
import statistics
import autosklearn.classification
from scipy import stats

In [27]:
#*** Example Problem ***
import sklearn.datasets
X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)

In [ ]:
#load and prepare dataset and labels
ae = keras.models.load_model("trained models/encoders/spec1")

img_res = 96
def load_image(dir):
	loaded_images = list()
	img_path = dir
	for filename in listdir(img_path):
		# load image
		img = Image.open(img_path+'/' + filename)
		if img.size[0] != img_res:
			img = img.resize((img_res,img_res))
		# img.show()
		gs_image = img.convert(mode='L') #convert to grayscale
		img_data = np.asarray(gs_image)
		img_data = img_data/255.0 #normalize

		# store loaded image
		loaded_images.append(img_data)

	number_of_instances = len(listdir(img_path))
	arr = np.asarray(loaded_images)
	arr = arr.reshape((number_of_instances, img_res, img_res, 1))
	return arr


X_train = load_image("temp comp (spectogram)/all spectogram/train")
X_test = load_image("temp comp (spectogram)/all spectogram/test")
pred1 = ae.predict(X_train)
pred2 = ae.predict(X_test)
encoded_X = np.concatenate((pred1, pred2), axis=0)

# import labels
lbl_pkl = joblib.load('train_label.pickle')
lbl_list = lbl_pkl.tolist()
Y_train = np.array(lbl_list)
lbl_pkl = joblib.load('test_label.pickle')
lbl_list = lbl_pkl.tolist()
Y_test = np.array(lbl_list)
y = np.concatenate((Y_train, Y_test), axis=0)

#convert 0 to 1, 1 to 0
y2 = []
for item in y:
	if item == 1.0:
		k = 0
	else:
		k = 1
	y2.append(k)
y = np.array(y2)

# train test split
X_train, X_test, Y_train, Y_test = train_test_split(encoded_X, y, test_size=0.20, random_state=1, shuffle=True)

In [ ]:
# Build and fit a classifier
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=120, per_run_time_limit=30)
automl.fit(X_train, y_train)
# automl.fit(X_train, y_train, dataset_name='breast_cancer')

In [ ]:
# Print the final ensemble constructed by auto-sklearn
print(automl.show_models())

In [39]:
# Get the prediction scores
predictions = automl.predict_proba(X_test)
# print(predictions)
predictions = np.ndarray.tolist(predictions)
pred_arr = []
for p in predictions:
  pred_arr.append(p[1])


In [50]:
#Get the BCE score and statistical values for predicted health indices
print("BCE score", sklearn.metrics.log_loss(y_test, pred_arr))
stats.describe(pred_arr)

BCE score 0.20389633713793381


DescribeResult(nobs=143, minmax=(0.10807033765053307, 0.9347086328462367), mean=0.6302230510616113, variance=0.12666068395709248, skewness=-0.5450872186423039, kurtosis=-1.584893114587342)